# Intro


Ponieważ jest to moje pierwsze zetknięcie z `NLP` i mam dość ograniczony czas, przyjąłem najprostsze podejście: `bag-of-words` + kilka komplementarnych i uniwersalnych algorytmów do klasyfikacji (model liniowy, svm, rf i boosting).  
Zastosowałem "zdroworozdądkowe" podejscie do features engineering (opisane w następnym akapicie). Generalnie chciałem oczyścić tekst z nieistotnych znaków, zaadresować problem literówek (nie rozwiązywany przez model języka którego użyłem) oraz ujednolicić formy wyrazów.  
Wyniki wszystkich zastosowanych przeze mnie modeli wskazywały na mocne przeuczenie, stąd decyzja o regularyzacji z wykorzystaniem optymalizacji hiperparametrów.  
W kolejnym kroku sprawdzę czy różne klasy modeli mylą się (`false negative` dla klas 1/2) na innych obserwacjach. Jeśli tak będzię zastosuję `blend` optymalnych klasyfikatorów. Wydaje się, że flagowanie tweetów klasą 1/2 jeśli którykowliek klasyfikator wskaże tą klasę może poprawić metryki.

W rozwiązaniu przyłożyłem większą niż zwykle wagę do aspektów Inzynierskich/MLOpsowych. Czytam teraz *Clean Code in Python* i zadanie wykorzystałem jako okazja do przećwiczenia tego czego się nauczyłem :) Mam nadzieję że nie przegiąłem z hintami :p Pomimo że zadanie jest z natury jednorazowe, starałem się wytworzyć kod który będzie czytelny, reużywalny i łatwo rozszeżalny.


**Pomysły jeśli miałbym więcej czsu**
* lepszy model języka. Obraz `Dockera` byłby cięższy i tagowanie mogłobybyć dłuższe, ale z przeprowadzonych testów wynika że w obu kwestiach mam duży zapas. 
* wykorzystanie 2/3-gramów
* wytworzenie nowych obserwacji dla klas 1/2.  
Pomysł który chciałbym przetestować to:
    * spośród `False Negative` dla tych klas wybrać n-gramy których częstość wystpępowania w kalsach 1/2 jest istotnie wyższa niż w klasie 0
    * te n-gramy tworzyłyby zbiór "rdzeni" nowego tweeta
    * dla każdego egzemplarza "rdzenia" dolosować n-gramy z typowych - których częstość występowania we wszystkich klasach jest podobna
    * zbiór takich tokenów tworzyłby nową obserwację  
    Parametry tej procedury można byłoby poddać optymalizacji (np. częstości, długość nowego tweeta, liczba "rdzeni" w nowym tweecie"
* generalnie w pracy nad danymi tabelarycznymi dużo poświęcam kwestiom `xai`, co pomaga zrozumieć działanie modeli a także ich poprawę i ciekawi mnie jak można byłoby te techniki wykorzystać w `NLP`
* wykorzystanie modeli deep learning dedykowanych `NLP`


W tym notebooku opisuję tylko kroki czysto ML-owe. Informacje o kwestiach inżynierskich zawarłem w pliku Readme.md

# setup

In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import os
import re
import ast
import spacy
import itertools
import pickle
import pandas as pd
import numpy as np

from random import choice
from typing import Any, List, Set, Dict, Tuple, Optional
from sklearn.metrics import multilabel_confusion_matrix
from hyperopt import hp

In [136]:
from src.preproc import DataReader
from src.models import Baseline, SVM, Ridge, RF, LGBM
from src.model_utils import model_performance, model_performance_summary, Optimizer, best_hyperopt_params, score

In [4]:
train_text = './data/raw/training_set_clean_only_text.txt'
train_tags = './data/raw/training_set_clean_only_tags.txt'
test_text = './data/raw/test_set_only_text.txt'
test_tags = './data/raw/test_set_only_tags.txt'

# czyszczenie danych + tokenizacja

**Czyszczenie danych**  
Wykonałem następujące kroki:

1. usunięcie frazy `@anonymized_account`
1. usunięcie znaków innych niż litery i spacje (znaki specjalne, emotikonki, etc.)  
Do ewentualnego rozważenia, wykorzystanie emotikonek. Być może, np.: występowanie obok siebie pistoletu i smutnej buzi niesie jakiś sygnał. Można byłoby to zrealizować zastąpieniem kodów emotek tokenami nie występującymi w korpusie z zadania.
1. usunięcie zduplikowanych spacji
1. sprowadzenie do małych liter
1. **tokenizacja**: wykorzytałem lematyzację z wykorzystaniem lekkiego modelu `pl_spacy_model`  
Znalazłem w międzyczasie lepsze modele, w szczególności takie które radzą sobie z literówkami. Jednak ze względu na ograniczenie czasowe zostałem przy podstawowym modelu.
1. usunięcie polskich znaków diakrytycznych
1. usunięcie zduplikowanych i sąsiadujących liter  
W obu powyższych krokach mam nadzieję zaadresować problem literówek.
1. oczyszczenie rozstrzelonych wyrazów, tj.: *s u p e r* -> *super*  
Warte byłoby też przetestowanie zastosowania trzech powyższych kroków przed lematyzacją, szczególnie jeśli korzystałbym z lepszego modelu języka
1. usunięcie tokenów 0/1 literowych

Ze względu na brak czasu powyższych kroków nie poddaje optymalizacji. Elementem każdego klasyfikatora są jeszcze dwa etapy (ich parametry będą optymalizowane).
1. `countVectorizer` - realizacja `bag-of-words`
1. `Tfidf` - zmniejszenie wagi tokenów z wysoką częstością - słów powszechnych które nie niosą informacji potrzebnej do klasyfikacji.


In [5]:
train_data = DataReader(train_text, train_tags)

INFO:io:reading from pickle
INFO:io:data ready


-------- stats --------
tweets count:
10041
tokens in tweet distribution:
count    10041.000000
mean         9.629320
std          4.282671
min          0.000000
25%          6.000000
50%          9.000000
75%         13.000000
95%         18.000000
99%         20.000000
max         28.000000
Name: tokens_count, dtype: float64
unique tokens:
14420
tags count:
0    9190
1     253
2     598
Name: tag, dtype: int64


In [6]:
test_data = DataReader(test_text, test_tags)

INFO:io:reading from pickle
INFO:io:data ready


-------- stats --------
tweets count:
1000
tokens in tweet distribution:
count    1000.000000
mean        9.445000
std         4.166093
min         2.000000
25%         6.000000
50%         9.000000
75%        12.000000
95%        18.000000
99%        19.000000
max        25.000000
Name: tokens_count, dtype: float64
unique tokens:
3087
tags count:
0    866
1     25
2    109
Name: tag, dtype: int64


**Obserwacje**:
* zbiór jest niezbalansowany - klasy 1/2 są żadsze (2.5% i 6% odpowiednio). To zachęca do przetestowania przetestowania algorytmów do detekcji anomalii. W mojej praktyce modele typu `isolation forest` lub `ecliptic envelope` w pierwszej warstwie modeli czasami pomagały poprawić jakość i stabilność całego stacku. Do zastosowania jeśli zostanie mi czas.
* zbiór testowy jest bardziej "rzadki": jest 3k unikalnych tokenów w 1k tweetów - stosunek ~3x, w zbiorze treningowym tylko 1.4x (przy podobnym rozkładzie liczby tokenów w tweet-cie)
* częstość klasy 2 `hate-speach` jest prawie dwókrotnie wyższa w zbiorze testowym
* te dwa fakty mogą świadczyć że zbiory treningowy i testowy pochodzą z różnych rozkładów (np.: zostały zebrane w różnych okresach czasu, lub wśród różnych grup użytkowników). Jeśli tak by było, trenowanie modeli które dobrze generalizowałyby się na zbiór testowy byłoby utrudnione.

# modele na domyślnych parametrach

Wytrenuję teraz modele z domyślnymi ustawieniami oraz model "bazowy" - prognozujący zawsze klasę 0.

In [7]:
# prep data
X = train_data.df['tokens']
y = train_data.df['tag']

X_test = test_data.df['tokens']
y_test = test_data.df['tag']

In [147]:
# train models
base_results = model_performance(Baseline(), X, y, X_test, y_test)
svm_results = model_performance(SVM(), X, y, X_test, y_test)
ridge_results = model_performance(Ridge(), X, y, X_test, y_test)
rf_results = model_performance(RF(), X, y, X_test, y_test)
lgbm_results = model_performance(LGBM(), X, y, X_test, y_test)


default_results = [base_results, svm_results, ridge_results, rf_results, lgbm_results]

In [83]:
# display results
for model_results in default_results:
    model_performance_summary(model_results)


Model: baseline (Baseline)
                macro	micro
score train:	0.9152	0.3186
score oof:	0.9152	0.3186
score test:	0.866	0.3094

Model: default_svm (SVM)
                macro	micro
score train:	0.9907	0.9538
score oof:	0.9282	0.5141
score test:	0.878	0.4089

Model: default_ridge (Ridge)
                macro	micro
score train:	0.9594	0.7345
score oof:	0.9239	0.4439
score test:	0.874	0.3616

Model: default_rf (RF)
                macro	micro
score train:	0.9997	0.998
score oof:	0.9271	0.4766
score test:	0.868	0.3435

Model: default_rf (LGBM)
                macro	micro
score train:	0.958	0.7874
score oof:	0.9169	0.3997
score test:	0.866	0.3431


**Obserwacje**:
* model `baseline` działa bardzo dobrze jeśli chodzi o `f1 macro`. Wynika to z faktu, że zbiór jest niezbalansowany.
* wydaje się że wybór `f1 macro` jako głównej metryki jest wątpliwy w tym przypadku. Kluczowe jest flagowanie klas 1/2, ponieważ koszt `False Negative` dla nich jest znacznie wyższy.  
Modele dostrojone pod `f1 macro` mogą nie być optymalne do tego zadania. Nie mniej zostaję przy zadanej metryce.
* wszystkie modele mocno przeuczają - jest wyraźny spadek pomiędzy `score oof` i ` score train`. W następnym kroku postaram się zregularyzować je przez optymalizację hiperparametrów.
* fakt, że `score test` jest systematycnie niższy niż `score oof` też przemawia za hipotezą, że zbiory pochodzą z innego rozkładu

Sprawdzę teraz wyniki blizej.

In [10]:
# conf matrix
for model_results in default_results:
    model_performance_summary(model_results, scores=False, matrix=True)


------------------------------------------------------
Model: baseline (Baseline)

train conf matrix


pred,0
act,
0,9190
1,253
2,598



oof conf matrix


pred,0
act,
0,9190
1,253
2,598



test conf matrix


...

pred,0,1,2
act,,,
0,9183.0,1.0,6.0
1,102.0,150.0,1.0
2,312.0,0.0,286.0



oof conf matrix


pred,0,1,2
act,,,
0,9134,12,44
1,234,9,10
2,521,13,64



test conf matrix


pred,0,1,2
act,,,
0,860.0,3.0,3.0
1,25.0,0.0,0.0
2,103.0,0.0,6.0


**Obserwacje**:
* wszystkie modele nie radzą sobie w generalizacją, nie są na tyle dobre w wykrywaniu 1/2 żeby mieć zastosowanie produkcyjne.
* `svm` wygląda najlepiej, chociaż myli się pomiędzy klasami 1 i 2
* `ridge` jest niedouczony - nawet na zbiorze testowym jest dużo `false negative` dla klas 1/2. Być może zmniejszenie domyślnej regularyzacji mu pomoże. Generalnie chciałbym tu zastosować `lasso` szczególnie dla selekcji nieistotnych ficzerów ale nie znalazłem na szybko algorytmu działającego na reprezentacji rzadkiej
* `rf` nauczył się zbioru treningowego doskonale :) Ale generalizuje się słabo, na test prawie nic nie wykrył (kolejna przesłanka za hipotezą o różnym rozkładzie train i test). Tu będzie potrzebna mocna regularyzacja
* `gbm` w przeciwieństwie do `rf` nie nauczył się wszystkiego, generalizuje się słabiej.

Na podstawie powyższych skupię się na modelach w takiej kolejności `svm` > `rf` > `gbm` > `ridge`.

# optymalizacja hiperparametrów

Do optymalizacji hiperparametrów skorzystam z biblioteki `hyperopt`.  
Ma ona tą przewagę na standardowym `grid/random search`, że aktywnie wykorzystuje wyniki poprzednich kroków do wyboru kolejnego punktu hiperprzestrzeni parametrów do przetestowania.  
Pozwala to znacznie skrócić czas potrzebny na znalezienie optymalnych parametrów, a także przeszukiwać większe przestrzenie.  

W losowych krokach optymalizacji dodaję losowy wybór punktu przestrzeni. Ma to zapobiec utknięciu optymalizatora w lokalnym minimum.

Jako funkcję do optymalizacji przyjmę `f1 makro` na prognozach `oof`. Dodatkowo zależy mi na stabilności wyników, dlatego dodam małą karę do funkcji kosztu za rozstrzał metryki pomiędzy train i oof.  

Gdyby metryka zadania nie była zadana wybrałbym `f1 mikro` lub `f1 mikro` liczoną tylko na klasach 1/2. Ewentualnie zwiększyłbym wagę `recall` (np. `f2 score`) ponieważ uważam, że koszt `Fals Negative` jest większy niż `Fals Positive`.

Zapisuję statystyki ewaluacji każdego punktu w przestrzeni. Przy większej ilości czasu przeprowadziłbym analizę wpływu hiperparametrów na jakość modeli, to pozwoliłoby zrozumieć wpływ każdego z parametrów na metryki a także lepiej zdefiniować zakres przestrzeni hiperparametrów.  
Sprawdzę też stabilność najlepszych puntków i wybiorę ten punkt którego wyniki są najbardziej stabilne. Wykorzystam do tego powtórzoną walidację krzyżową ze zmiennym ziarnem podziału zbiory na foldy.

Do optymalizacji parametrów używam tylko zbioru train. Dla każdego kalsyfikatora testowanych będzie 500 punktów z przestrzeni.

Najpierw zdefiniuję przestrzeń dla transformerów - uniwersalną dla wszystkich klasyfikatorów.

In [230]:
hyper_space_transformers = {
    'vect__binary': hp.choice('binary', [True, False]),
    
    'tfidf__norm': hp.choice('norm', ['l1', 'l2']), # norm type for output vector. l2 - sum of squares, l1 - absolute values
    'tfidf__use_idf': hp.choice('use_idf', [True, False]),    
    'tfidf__smooth_idf': hp.choice('smooth_idf', [True, False]),    
    'tfidf__sublinear_tf': hp.choice('sublinear_tf', [True, False]),
}

## svm

In [282]:
hyper_space_clf = {
    'clf__dual': hp.choice('dual', [True, False]),
    'clf__C': hp.uniform('C', 0.0001, 10),
    'clf__loss': hp.choice('loss', ['squared_hinge']),
    'clf__class_weight': hp.choice('class_weight', ['balanced', None]),
    'clf__max_iter': hp.uniform('max_iter', 1000, 5000),
    'clf__intercept_scaling': hp.uniform('intercept_scaling', 0.1, 10),
}

hyper_space = {**hyper_space_transformers, **hyper_space_clf}

In [283]:
opt = Optimizer()
opt.optimize(SVM, X, y, param_space = hyper_space, n_rounds = 500)

opt_results_svm = opt.results

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.003541 seconds
INFO:hyperopt.tpe:TPE using 0 trials
/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase 

  2%|▏         | 10/500 [01:16<1:02:04,  7.60s/trial, best loss: -0.9163899999999999]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.004990 seconds
INFO:hyperopt.tpe:TPE using 10/10 trials with best loss -0.916390
/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear 

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.br

...

 96%|█████████▌| 480/500 [43:45<01:47,  5.39s/trial, best loss: -0.92161]

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

INFO:hyperopt.tpe:build_posterior_wrapper took 0.003028 seconds
INFO:hyperopt.tpe:TPE using 480/480 trials with best loss -0.921610
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003555 seconds
INFO:hyperopt.tpe:TPE using 481/481 trials with best loss -0.921610
/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

 98%|█████████▊| 490/500 [44:16<00:46,  4.69s/trial, best loss: -0.92161]

/Users/michal.bronis/Desktop/nlp/poleval19/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "

INFO:hyperopt.tpe:build_posterior_wrapper took 0.002959 seconds
INFO:hyperopt.tpe:TPE using 490/490 trials with best loss -0.921610
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003337 seconds
INFO:hyperopt.tpe:TPE using 491/491 trials with best loss -0.921610
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003259 seconds
INFO:hyperopt.tpe:TPE using 492/492 trials with best loss -0.921610
INFO:hyperopt.tpe:build_posterior_wrapper took 0.002867 seconds
INFO:hyperopt.tpe:TPE using 493/493 trials with best loss -0.921610
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003179 seconds
INFO:hyperopt.tpe:TPE using 494/494 trials with best loss -0.921610
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003371 seconds
INFO:hyperopt.tpe:TPE using 49

100%|██████████| 500/500 [44:38<00:00,  5.36s/trial, best loss: -0.92161]


## rf

In [83]:
hyper_space_clf = {
    'clf__n_estimators': hp.uniform('n_estimators', 10, 200),
    'clf__max_depth': hp.uniform('max_depth', 2, 100),
    'clf__min_samples_leaf': hp.uniform('min_samples_leaf', 1, 20),
    'clf__criterion': hp.choice('criterion', ['gini', 'entropy']),
    'clf__max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
    'clf__max_leaf_nodes': hp.uniform('max_leaf_nodes', 4, 50),
    'clf__bootstrap': hp.choice('bootstrap', [True, False]),
    'clf__n_jobs': hp.choice('n_jobs', [-1]),
    'clf__class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample', None]),
    'clf__max_samples': hp.uniform('max_samples', 0.2, 1),
}

hyper_space = {**hyper_space_transformers, **hyper_space_clf}

In [84]:
opt = Optimizer()
opt.optimize(RF, X, y, param_space = hyper_space, n_rounds = 500)

opt_results_rf = opt.results

  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.006809 seconds
INFO:hyperopt.tpe:TPE using 0 trials


  1%|          | 10/1000 [01:17<2:07:25,  7.72s/trial, best loss: -0.91535]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.004026 seconds
INFO:hyperopt.tpe:TPE using 10/10 trials with best loss -0.915350


  3%|▎         | 30/1000 [02:48<1:28:03,  5.45s/trial, best loss: -0.91541]

...

 95%|█████████▌| 950/1000 [1:32:03<05:30,  6.61s/trial, best loss: -0.91622]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.003927 seconds
INFO:hyperopt.tpe:TPE using 950/950 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004018 seconds
INFO:hyperopt.tpe:TPE using 951/951 trials with best loss -0.916220


 97%|█████████▋| 970/1000 [1:34:15<03:20,  6.69s/trial, best loss: -0.91622]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.004588 seconds
INFO:hyperopt.tpe:TPE using 970/970 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004644 seconds
INFO:hyperopt.tpe:TPE using 971/971 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004684 seconds
INFO:hyperopt.tpe:TPE using 972/972 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004292 seconds
INFO:hyperopt.tpe:TPE using 973/973 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004034 seconds
INFO:hyperopt.tpe:TPE using 974/974 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004239 seconds
INFO:hyperopt.tpe:TPE using 975/975 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003927 seconds
INFO:hyperopt.tpe:TPE using 976/976 trials with best loss -0.916220
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004019 seconds
INFO:hyperop

100%|██████████| 1000/1000 [1:37:37<00:00,  5.86s/trial, best loss: -0.91622]


## gbm

In [92]:
hyper_space_clf = {
    'clf__n_estimators': hp.quniform('n_estimators', 50, 1000, 1),
    'clf__linear_tree': hp.choice('linear_tree', [True, False]),
    'clf__objective': hp.choice('objective', ['softmax', 'ova']),
    'clf__boosting': hp.choice('boosting', ['gbdt', 'rf', 'dart', 'goss']),
    'clf__learning_rate': hp.uniform('learning_rate', 0.005, 0.2),
    'clf__num_leaves': hp.quniform('num_leaves', 5, 50, 1),
    'clf__max_depth': hp.quniform('max_depth', 2, 6, 1),
    'clf__min_data_in_leaf': hp.uniform('min_data_in_leaf', 1, 20),
    'clf__feature_fraction': hp.uniform('feature_fraction', 0.3, 1.0),
    'clf__lambda_l1': hp.uniform('lambda_l1', 0.0, 10.0),
    'clf__lambda_l2': hp.uniform('lambda_l2', 0.0, 10.0),
    'clf__path_smooth': hp.uniform('path_smooth', 0.0, 10.0),
}

hyper_space = {**hyper_space_transformers, **hyper_space_clf}

In [ ]:
opt = Optimizer()
opt.optimize(LGBM, X, y, param_space = hyper_space, n_rounds = 500)

opt_results_gbm = opt.results

  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.007752 seconds
INFO:hyperopt.tpe:TPE using 0 trials


[LightGBM] [Warning] lambda_l1 is set=4.364991663822489, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.364991663822489
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.761499459476483, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.761499459476483
[LightGBM] [Warning] lambda_l2 is set=8.144005099567996, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.144005099567996
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] lambda_l1 is set=4.364991663822489, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.364991663822489
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] feature_fraction is set=0.761499459476483, colsample_bytree=1.0 will be ignored. 

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] lambda_l1 is set=3.42740983741146, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.42740983741146
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] feature_fraction is set=0.30399762687040205, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30399762687040205
[LightGBM] [Warning] lambda_l2 is set=7.667115111712377, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.667115111712377
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] lambda_l1 is set=3.42740983741146, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.42740983741146
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_le

[LightGBM] [Warning] lambda_l2 is set=8.517294707452406, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.517294707452406
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=5.967365665448289, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.967365665448289
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] feature_fraction is set=0.974313048189164, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.974313048189164
[LightGBM] [Warning] lambda_l2 is set=8.517294707452406, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.517294707452406
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=5.967365665448289, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] lambda_l1 is set=1.3496913996804705, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3496913996804705
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] feature_fraction is set=0.7405844986542919, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7405844986542919
[LightGBM] [Warning] lambda_l2 is set=0.7041795816305174, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7041795816305174
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] lambda_l1 is set=1.3496913996804705, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3496913996804705
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_

[LightGBM] [Warning] lambda_l1 is set=0.17329120268416598, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.17329120268416598
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.4213459535620334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4213459535620334
[LightGBM] [Warning] lambda_l2 is set=7.1442586220959985, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.1442586220959985
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] lambda_l1 is set=0.17329120268416598, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.17329120268416598
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.4213459535620334, colsample_bytree=1.0 will be 

INFO:hyperopt.tpe:build_posterior_wrapper took 0.006536 seconds
INFO:hyperopt.tpe:TPE using 20/20 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.007754 seconds
INFO:hyperopt.tpe:TPE using 21/21 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.006811 seconds
INFO:hyperopt.tpe:TPE using 22/22 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.008111 seconds
INFO:hyperopt.tpe:TPE using 23/23 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.006622 seconds
INFO:hyperopt.tpe:TPE using 24/24 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.006039 seconds
INFO:hyperopt.tpe:TPE using 25/25 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.004941 seconds
INFO:hyperopt.tpe:TPE using 26/26 trials with best loss -0.915990
INFO:hyperopt.tpe:build_posterior_wrapper took 0.008167 seconds
INFO:hyperopt.tpe:TPE usin

[LightGBM] [Warning] lambda_l1 is set=0.2482172700090861, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2482172700090861
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] feature_fraction is set=0.4816644059870498, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4816644059870498
[LightGBM] [Warning] lambda_l2 is set=3.188204259850385, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.188204259850385
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.2482172700090861, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2482172700090861
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] feature_fraction is set=0.4816644059870498, colsample_bytree=1.0 will be ig

[LightGBM] [Warning] lambda_l2 is set=9.66737707791545, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.66737707791545
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=2.7584383765463127, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.7584383765463127
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2
[LightGBM] [Warning] feature_fraction is set=0.8499202264307988, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8499202264307988
[LightGBM] [Warning] lambda_l2 is set=9.66737707791545, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.66737707791545
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=2.7584383765463127, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.7

## ridge

In [92]:
hyper_space_clf = {
    'clf__alpha': hp.uniform('alpha', 0.1, 10.0),
    'clf__fit_intercept': hp.choice('fit_intercept', [True, False]),
    'clf__normalize': hp.choice('normalize', [True, False]),
    'clf__tol': hp.uniform('tol', 0.0001, 0.1),
}

hyper_space = {**hyper_space_transformers, **hyper_space_clf}

In [312]:
opt = Optimizer()
opt.optimize(Ridge, X, y, param_space = hyper_space, n_rounds = 500)

opt_results_ridge = opt.results

# wybór optymalnych parametrów

Chcę upewnić się co do stabilności znalezionych parametrów.
Dla każdego z nalepszych 10 punktów hiperprzestrzeni przeprowadzę 50x walidację krzyżową, za każdym razem z innym ziarnem dla splitów. Jako ostateczny zestaw parametrów wybiorę ten z najwyższą medianą `f1 makro`.  
Przy wyborze można ewentualnie wziąć też pod uwagę wariancję wyników. Im mniejsza wariancja tym lepszy model.

In [316]:
best_params_svm = best_hyperopt_params(SVM, X, y, opt_results_svm)
best_params_rf = best_hyperopt_params(RF, X, y, opt_results_rf)
best_params_gbm = best_hyperopt_params(LGBM, X, y, opt_results_gbm)
best_params_ridge = best_hyperopt_params(Ridge, X, y, opt_results_ridge)

In [272]:
# train models with opt params
svm_results_opt = model_performance(SVM(best_params_svm), X, y, X_test, y_test)
rf_results_opt = model_performance(RF(best_params_rf), X, y, X_test, y_test)
lgbm_results_opt = model_performance(LGBM(best_params_gbm), X, y, X_test, y_test)
ridge_results_opt = model_performance(Ridge(best_params_ridge), X, y, X_test, y_test)


opt_results = [svm_results_opt, rf_results_opt, lgbm_results_opt, ridge_results_opt]

[LightGBM] [Warning] lambda_l1 is set=6.452187474071411, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.452187474071411
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.38382875426001184, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.38382875426001184
[LightGBM] [Warning] lambda_l2 is set=9.670834661974457, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.670834661974457
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=6.452187474071411, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.452187474071411
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.38382875426001184, colsample_bytree=1.0 will be ign

In [318]:
# display results
for model_results in opt_results:
    model_performance_summary(model_results)


Model: default_svm (SVM)
                macro	micro
score train:	0.993	0.9664
score oof:	0.9281	0.5169
score test:	0.876	0.4026

Model: default_rf (RF)
                macro	micro
score train:	0.9165	0.335
score oof:	0.9156	0.3242
score test:	0.869	0.3278

Model: default_gbm (LGBM)
                macro	micro
score train:	0.9276	0.4383
score oof:	0.9185	0.3729
score test:	0.873	0.3555

Model: default_ridge (Ridge)
                macro	micro
score train:	0.988	0.9382
score oof:	0.9282	0.5061
score test:	0.873	0.387


**Obserwacje**
* optymalizacja parametrów nie pozwoliła istotnie poprawić prognoz modeli
* w niektórych przypadkach, najlepszy znaleziony punkt nie był lepszy od default-owego.

# modele z optymalnymi parametrami

In [320]:
# train and save best models
final_svm = SVM(model_name='final_svm')
final_rf = RF(model_name='final_rf')
final_gbm = LGBM(best_params_gbm, model_name='final_gbm')
final_ridge = Ridge(best_params_ridge, model_name='final_ridge')

svm_results_final = model_performance(final_svm, X, y, X_test, y_test, save_model=True)
rf_results_final = model_performance(final_rf, X, y, X_test, y_test, save_model=True)
lgbm_results_final = model_performance(final_gbm, X, y, X_test, y_test, save_model=True)
ridge_results_final = model_performance(final_ridge, X, y, X_test, y_test, save_model=True)

[LightGBM] [Warning] lambda_l1 is set=6.452187474071411, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.452187474071411
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.38382875426001184, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.38382875426001184
[LightGBM] [Warning] lambda_l2 is set=9.670834661974457, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.670834661974457
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=6.452187474071411, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.452187474071411
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.38382875426001184, colsample_bytree=1.0 will be ign

# analiza błędnych klasyfikacji

Przeprowadzę teraz analizę błędnych klasyfikacji. Interesuje mnie czy klasyfikatory mylą się na tych samych czy różnych tweet-ach.  
W pierwszym przypadku jako ostateczny klasyfikator wybiorę ten z najlepszym scorem.  
W drugim zastosuję blend.

In [321]:
svm_oof = svm_results_final.tags_oof_pred
rf_oof = rf_results_final.tags_oof_pred
gbm_oof = lgbm_results_final.tags_oof_pred
ridge_oof = ridge_results_final.tags_oof_pred

In [323]:
results = pd.DataFrame(zip(y, svm_oof, rf_oof, ridge_oof, gbm_oof), 
                       columns=['act', 'svm', 'rf', 'ridge', 'lgbm'])

results['svm'] = (results['svm']==results['act']).astype(int)
results['rf'] = (results['rf']==results['act']).astype(int)
results['ridge'] = (results['ridge']==results['act']).astype(int)
results['lgbm'] = (results['lgbm']==results['act']).astype(int)

results['counts_models_match'] = results[['svm', 'rf', 'ridge', 'lgbm']].sum(axis=1)

results = results.groupby(['act', 'counts_models_match']).size().reset_index().rename(columns={0:'count'})
results.pivot(index='act', columns='counts_models_match', values='count').fillna(0).astype(int)

counts_models_match,0,1,2,3,4
act,,,,,
0,3,24,41,40,9082
1,230,4,7,11,1
2,376,30,52,115,25


**Obserwacje**
* istotna część poprawnie znalezionych egzemplarzy klas 1/2 została oznaczona tylko przez jeden klasyfikator
* analiza wyników blendów prognoz pozwoli ustalić czy jest jeden dominujący klasyfikator czy są one komplementarne

# blend prognoz modeli

In [324]:
blend_base = pd.DataFrame(zip(svm_oof, rf_oof, ridge_oof, gbm_oof), columns=['svm',  'rf', 'ridge', 'lgbm'])

blend_permutations = list(itertools.product([0, 1], repeat=blend_base.shape[1]))
blend_permutations

blend_names = []
blend_tags = []

print(f"{'blend'+'f1makro'.rjust(21)}    mikro")
for p in blend_permutations:
    blend_name = ('svm_' if p[0]==1 else '') + ('rf_' if p[1]==1 else '') + ('ridge_' if p[2]==1 else '') + ('lgbm' if p[3]==1 else '')
    blend_names.append(blend_name)
    
    blend = blend_base.apply(
        lambda x: 
        1 if 1 in [p[0]*x[0], p[1]*x[1], p[2]*x[2]] else 
        2 if 2 in [p[0]*x[0], p[1]*x[1], p[2]*x[2]] else 0, axis=1)
    blend_tags.append(blend)
    
    print(f"{blend_name.ljust(18)}: {score(y, blend)}")

blend              f1makro    mikro
                  : (0.9152, 0.3186)
lgbm              : (0.9152, 0.3186)
ridge_            : (0.9282, 0.5061)
ridge_lgbm        : (0.9282, 0.5061)
rf_               : (0.9287, 0.484)
rf_lgbm           : (0.9287, 0.484)
rf_ridge_         : (0.9289, 0.5168)
rf_ridge_lgbm     : (0.9289, 0.5168)
svm_              : (0.9282, 0.5141)
svm_lgbm          : (0.9282, 0.5141)
svm_ridge_        : (0.9282, 0.5199)
svm_ridge_lgbm    : (0.9282, 0.5199)
svm_rf_           : (0.9284, 0.5192)
svm_rf_lgbm       : (0.9284, 0.5192)
svm_rf_ridge_     : (0.928, 0.5215)
svm_rf_ridge_lgbm : (0.928, 0.5215)


**Obserwacje**
* generalnie modele i ich blendy bardzo słabo radzą sobie z wykrywaniem klas 1/2, większość przypadków nie została poprawnie zaklasyfikowana.
* najlepszym pojedynczym klasyfikatorem w sensie `f1-macro` jest `RF`
* najlepszym blendem jest `RF`+`Ridge`, ten blend zostanie klasyfikatorem w aplikacji
* Gdybym miał decydować o produkcyjnym wykorzystaniu modelu wybrałbym jednak `SVM`+`Ridge`. Maksymalizuje `f1-micro` - najlepiej radzi sobie z zadaniem klasyfikacji klas 1/2.

# sprawdzenie na zbiorze testowym

In [329]:
# load final blend models
with open('./models/rf/final_rf.pkl', 'rb') as f:
    final_rf, _ = pickle.load(f)
    
with open('./models/ridge/final_ridge.pkl', 'rb') as f:
    final_ridge, _ = pickle.load(f)

In [337]:
test_rf_tags = final_rf.predict(X_test)
test_ridge_tags = final_ridge.predict(X_test)

test_tags = [1 if 1 in x else (2 if 2 in x else 0) for x in zip(test_rf_tags, test_ridge_tags)]

In [343]:
final_score = score(y_test, test_tags)
print(f"final blend f1-macro: {final_score[0]}, f1-micro: {final_score[1]}")

final blend f1-macro: 0.874, f1-micro: 0.4047
